In [1]:
# import all the packages needed
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np

In [2]:
# Project path
## Please change it to the location of your choice
%cd "/Users/chnam/OneDrive - UBC/DSCI_525/milestone_1"

/Users/chnam/Library/CloudStorage/OneDrive-UBC/DSCI_525/milestone_1


In [3]:
# Metadata
## Article ID of figshare
article_id = 14096681  
# Beuzen, Tomas (2021): Daily rainfall over NSW, Australia. figshare. Dataset. https://doi.org/10.6084/m9.figshare.14096681.v3

## URL
url = f"https://api.figshare.com/v2/articles/{article_id}"

## Header
headers = {"Content-Type": "application/json"}

In [4]:
# Change the output directory for the data files
output_directory = "rainforest/" 

In [5]:
# Send a GET request to list the available files
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data
files = data["files"]             # this is just the data about the files
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [7]:
%%time
# Download data.zip
files_to_dl = ["data.zip"]  # data.zip
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 5.17 s, sys: 9 s, total: 14.2 s
Wall time: 13min 21s


In [8]:
%%time
# Extract data.zip
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 7.35 s, sys: 785 ms, total: 8.14 s
Wall time: 8.36 s


In [9]:
# List the folder
%ls -ltr rainforest

total 24696800
drwxr-xr-x  30 chnam  staff         960 Mar 28 14:33 __MACOSX/
-rw-r--r--   1 chnam  staff  6475225405 Mar 28 14:50 combined.csv
-rw-r--r--   1 chnam  staff   814041183 Mar 28 16:19 data.zip
-rw-r--r--   1 chnam  staff    95376895 Mar 28 16:19 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--@  1 chnam  staff    94960113 Mar 28 16:19 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--@  1 chnam  staff    82474546 Mar 28 16:19 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--@  1 chnam  staff   127613760 Mar 28 16:19 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--@  1 chnam  staff   232118894 Mar 28 16:19 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--@  1 chnam  staff   330360682 Mar 28 16:19 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--@  1 chnam  staff   254009247 Mar 28 16:19 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--@  1 chnam  staff   235661418 Mar 28 16:19 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--@  1 chnam  staff   294260911 Mar 28 16:19 BCC-CSM2-MR_daily_rainfall_NSW.csv


In [10]:
# Show a list of paths
files = glob.glob('rainforest/*.csv')
files

['rainforest/MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv',
 'rainforest/AWI-ESM-1-1-LR_daily_rainfall_NSW.csv',
 'rainforest/NorESM2-LM_daily_rainfall_NSW.csv',
 'rainforest/ACCESS-CM2_daily_rainfall_NSW.csv',
 'rainforest/FGOALS-f3-L_daily_rainfall_NSW.csv',
 'rainforest/CMCC-CM2-HR4_daily_rainfall_NSW.csv',
 'rainforest/MRI-ESM2-0_daily_rainfall_NSW.csv',
 'rainforest/GFDL-CM4_daily_rainfall_NSW.csv',
 'rainforest/BCC-CSM2-MR_daily_rainfall_NSW.csv',
 'rainforest/EC-Earth3-Veg-LR_daily_rainfall_NSW.csv',
 'rainforest/CMCC-ESM2_daily_rainfall_NSW.csv',
 'rainforest/NESM3_daily_rainfall_NSW.csv',
 'rainforest/MPI-ESM1-2-LR_daily_rainfall_NSW.csv',
 'rainforest/ACCESS-ESM1-5_daily_rainfall_NSW.csv',
 'rainforest/FGOALS-g3_daily_rainfall_NSW.csv',
 'rainforest/INM-CM4-8_daily_rainfall_NSW.csv',
 'rainforest/MPI-ESM1-2-HR_daily_rainfall_NSW.csv',
 'rainforest/TaiESM1_daily_rainfall_NSW.csv',
 'rainforest/NorESM2-MM_daily_rainfall_NSW.csv',
 'rainforest/CMCC-CM2-SR5_daily_rainfall_NSW.csv',
 'r

In [17]:
# Show the columns available in a data file
sample1 = pd.read_csv('rainforest/SAM0-UNICON_daily_rainfall_NSW.csv')
cols = sample1.columns.to_list()
cols

['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']

In [33]:
%%time
## Combine data CSVs into a single CSV using pandas
files = glob.glob('rainforest/*.csv')
excluded_files = ['rainforest/observed_daily_rainfall_SYD.csv', 'rainforest/combined.csv']

df = pd.concat((pd.read_csv(file,
                            index_col=False, 
                            usecols=cols,
                            # low_memory = False
                           )
                .assign(model=re.findall('/([^_]*)', file)[0]) # find the model name before the first underscore and assign it to "model"
                for file in files if file not in excluded_files # do not include observed_daily_rainfall_SYD.csv and combined.csv
               )
              )
df.to_csv("rainforest/combined.csv")

CPU times: user 3min 29s, sys: 11.5 s, total: 3min 40s
Wall time: 3min 41s


In [34]:
df.shape

(62467843, 7)

In [35]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [36]:
# Convert to float32
df32 = df.astype('float32', errors='ignore')

In [37]:
# Check memory usage
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df32.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 3997.94 MB
Memory usage with float32: 2748.59 MB


In [149]:
%%time
# EDA (load data - select columns)
use_cols = ['rain (mm/day)', 'model']

df = pd.read_csv("rainforest/combined.csv",
                   usecols=use_cols)

CPU times: user 16.3 s, sys: 1.6 s, total: 17.9 s
Wall time: 18 s


In [150]:
%%time
# EDA (get mean, max, min, median by model)
df.groupby('model')['rain (mm/day)'].agg(['mean', 'max', 'min', 'median'])

CPU times: user 2.7 s, sys: 235 ms, total: 2.93 s
Wall time: 2.93 s


,mean,max,min,median
model,,,,
ACCESS-CM2,1.787025,432.939515,0.000000e+00,1.002723e-01
ACCESS-ESM1-5,2.217501,202.028315,-3.049319e-18,1.345675e-01
AWI-ESM-1-1-LR,2.026071,89.465749,9.065210e-14,4.426399e-02
BCC-CSM2-MR,1.951832,286.226671,3.766800e-24,4.321701e-02
BCC-ESM1,1.811032,164.058595,9.260348e-25,1.091290e-02
CMCC-CM2-HR4,2.279350,224.154387,0.000000e+00,2.687256e-01
CMCC-CM2-SR5,2.383389,202.845832,-4.077885e-13,2.171495e-01
CMCC-ESM2,2.266125,230.226552,-7.874900e-14,2.028418e-01
CanESM5,1.894328,173.812797,0.000000e+00,8.639222e-02


In [151]:
%%time
# EDA (load data - change dtype)
use_cols = ['rain (mm/day)', 'model']

df32 = pd.read_csv("rainforest/combined.csv",
                   dtype={'lat_min':'float32',
                          'lat_max':'float32',
                          'lon_min':'float32',
                          'lon_max':'float32',
                          'rain (mm/day)':'float32'})

CPU times: user 29.5 s, sys: 6.72 s, total: 36.2 s
Wall time: 37.7 s


In [152]:
%%time
# EDA (.describe())
df32.describe()

CPU times: user 6.86 s, sys: 1.42 s, total: 8.28 s
Wall time: 8.48 s


,Unnamed: 0,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,1.479732e+06,-3.310497e+01,-3.197765e+01,1.469057e+02,1.482150e+02,1.901173e+00
std,1.070480e+06,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,0.000000e+00,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,5.805280e+05,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,1.267234e+06,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,2.273266e+06,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,5.154239e+06,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [153]:
# import for R. Please change the path by referring to the result of `conda run -n 525_2023 python -m site`
import os
os.environ['R_HOME'] = '/opt/miniconda3/envs/525/lib/R'

In [154]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [155]:
# Please change the path for the data files
%cd /Users/chnam/OneDrive - UBC/DSCI_525/milestone_1/

/Users/chnam/Library/CloudStorage/OneDrive-UBC/DSCI_525/milestone_1


In [156]:
%%R
suppressMessages(library(dplyr))
suppressMessages(library(readr))

In [157]:
# Use R and python interchangeably (with arrow)
import pyarrow.dataset as ds
import rpy2_arrow.pyarrow_rarrow as pyra

In [158]:
%%time
dataset = ds.dataset("rainforest/combined.csv", format="csv")

# Converting the `pyarrow dataset` to a `pyarrow table`
table = dataset.to_table()

# Converting a `pyarrow table` to a `rarrow table`
r_table = pyra.converter.py2rpy(table)

CPU times: user 12.1 s, sys: 2.08 s, total: 14.2 s
Wall time: 19 s


In [161]:
%%R -i r_table
# EDA (count by model)
start_time <- Sys.time()
result <- r_table %>%
    select(model) %>%
    count(model)
end_time <- Sys.time()
print(result %>% collect())
print(end_time - start_time)

# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 NorESM2-LM        919800
 4 ACCESS-CM2       1932840
 5 FGOALS-f3-L      3219300
 6 CMCC-CM2-HR4     3541230
 7 MRI-ESM2-0       3037320
 8 GFDL-CM4         3219300
 9 BCC-CSM2-MR      3035340
10 EC-Earth3-Veg-LR 3037320
# ℹ 17 more rows
# ℹ Use `print(n = ...)` to see more rows
Time difference of 0.02721 secs


In [162]:
%%R
# EDA (list records with NA `rain (mm/day)`)
start_time <- Sys.time()
result <- r_table %>%
    select(time, lat_min, lat_max, lon_min, lon_max, `rain (mm/day)`, model) %>%
    filter(is.na(`rain (mm/day)`))
end_time <- Sys.time()
print(result %>% collect())
print(end_time - start_time)

# A tibble: 3,219,300 × 7
   time                lat_min lat_max lon_min lon_max `rain (mm/day)` model    
   <dttm>                <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>    
 1 1913-09-19 04:00:00      NA     -36      NA    141.              NA GFDL-ESM4
 2 1913-09-20 04:00:00      NA     -36      NA    141.              NA GFDL-ESM4
 3 1913-09-21 04:00:00      NA     -36      NA    141.              NA GFDL-ESM4
 4 1913-09-22 04:00:00      NA     -36      NA    141.              NA GFDL-ESM4
 5 1913-09-23 04:00:00      NA     -36      NA    141.              NA GFDL-ESM4
 6 1913-09-24 04:00:00      NA     -36      NA    141.              NA GFDL-ESM4
 7 1913-09-25 04:00:00      NA     -36      NA    141.              NA GFDL-ESM4
 8 1913-09-26 04:00:00      NA     -36      NA    141.              NA GFDL-ESM4
 9 1913-09-27 04:00:00      NA     -36      NA    141.              NA GFDL-ESM4
10 1913-09-28 04:00:00      NA     -36      NA    141.              NA GFDL-ESM4
# 